In [55]:
import numpy as np
import pandas as pd
import pyvo
from astropy.io import ascii
from astropy.table import Table

In [34]:
service = pyvo.dal.TAPService('https://irsa.ipac.caltech.edu/TAP')
wise_2mass_jkw2_colorcut = service.submit_job("""
            SELECT ra, dec, j_m_2mass, k_m_2mass, w1mpro, w2mpro, w1snr, w2snr
            FROM allwise_p3as_psd
            WHERE k_m_2mass - w2mpro >= 1.8 - 0.848 * (j_m_2mass - k_m_2mass) AND
            j_m_2mass > 0 AND
            w1snr >= 5 AND
            w2snr >= 5
    """)

wise_2mass_jkw2_colorcut.run()

In [45]:
print(type(wise_2mass_jkw2_colorcut))

<class 'pyvo.dal.tap.AsyncTAPJob'>


In [48]:
table = wise_2mass_jkw2_colorcut.fetch_result()

In [58]:
table_vo = table.votable
table_vo.to_xml('wise_2mass_jkw2_colorcut.xml')

<class 'astropy.io.votable.tree.VOTableFile'>
